In [3]:
import sys
sys.path.insert(0, "../Src/")
import generateVoices as gv
import evaluateVoices as ev
import pandas as pd
from os.path import exists

%autosave 5

Autosaving every 5 seconds


In [ ]:
# df = pd.read_csv("../Data/test_data.csv")
# df

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
0,1.0,TEST,DR4,MGMM0,SX139.WAV,TEST/DR4/MGMM0/SX139.WAV,TEST\\DR4\\MGMM0\\SX139.WAV,False,True,False,False,False
1,2.0,TEST,DR4,MGMM0,SX139.WAV.wav,TEST/DR4/MGMM0/SX139.WAV.wav,TEST\\DR4\\MGMM0\\SX139.WAV.wav,True,True,False,False,False
2,3.0,TEST,DR4,MGMM0,SX139.TXT,TEST/DR4/MGMM0/SX139.TXT,TEST\\DR4\\MGMM0\\SX139.TXT,False,False,False,False,True
3,4.0,TEST,DR4,MGMM0,SI499.WRD,TEST/DR4/MGMM0/SI499.WRD,TEST\\DR4\\MGMM0\\SI499.WRD,False,False,True,False,False
4,5.0,TEST,DR4,MGMM0,SX319.WRD,TEST/DR4/MGMM0/SX319.WRD,TEST\\DR4\\MGMM0\\SX319.WRD,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
31673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def getBestModelName():
    wordErrorRateResults = pd.read_csv("../Data/wordErrorRateResults.csv")
    bestModel = wordErrorRateResults.sort_values(by="wordErrorRate").iloc[0]
    return bestModel["modelName"]

In [ ]:
def getSourceVoice(speaker,speakerDf):
    sourceVoiceFile = f"../Data/concatenatedInput/{speaker}_concatenated.wav"
    if exists(sourceVoiceFile):
        return sourceVoiceFile
    gv.concatenateAudio(speaker, speakerDf)
    if exists(sourceVoiceFile):
        return sourceVoiceFile
    else:
        raise FileNotFoundError(f"Source voice file for {speaker} not found after concatenation.")

In [ ]:
def getFADJson(datasetName):
    fadJson = []
    if exists(f"../Data/fad_{datasetName}.json"):
        with open(f"../Data/fad_{datasetName}.json", "r") as f:
            fadJson = gv.loadJson(f)
    return fadJson

In [ ]:
def createFakeAudio(speakerDf, sourceVoice, modelName, datasetName):
    fadJson = getFADJson(datasetName)
    textFiles = speakerDf[speakerDf['path_from_data_dir'].str.contains('.TXT', na=False)]
    for textFile in textFiles:
        fakeAudioFile = f"../Data/fakeAudio/{datasetName}/{textFile['path_from_data_dir'].replace('.TXT', '.wav')}"
        if exists(fakeAudioFile):
            print(f"Fake audio file {fakeAudioFile} already exists.")
            continue
        


In [ ]:
def prepareFADData(modelName,datasetName):
    print(f"Creating voices for {modelName} on {datasetName} dataset")
    
    df = gv.readCsv(datasetName)
    speakers = gv.getSpeakers(df)
    for speaker in speakers:
        speakerDf = gv.getFilesBySpeaker(df, speaker)
        sourceVoice = getSourceVoice(speaker, speakerDf)
        createFakeAudio(speakerDf, sourceVoice, modelName, datasetName)
        

In [ ]:
def main():
    bestModelName = getBestModelName()
    prepareFADData(bestModelName, 'train')
    prepareFADData(bestModelName, 'test')
    print('done')

In [ ]:
if __name__ == '__main__':
    main()